In [3]:
import cv2
import json
import numpy as np
import os

In [2]:
def convert_seconds_to_timestamp(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

In [12]:
def detect_logo_in_video(video_path, logo_path, threshold=0.6):
    # Abrir o vídeo e a logotipo target
    print(video_path)
    cap = cv2.VideoCapture(video_path)
    logo = cv2.imread(logo_path, 0)  # Lendo a logotipo em escala de cinza

    # Verifica se o vídeo foi aberto corretamente
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    # Obter fps e calcular a duração do vídeo
    fps = cap.get(cv2.CAP_PROP_FPS)
    # frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # duration = frame_count / fps

    # Dimensões da logotipo
    # w, h = logo.shape[::-1]
    #print(w)
    #print(h)

    detection_started = False
    logo_timestamps = []
    start_time = None
    end_time = None

    frame_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Converter frame para escala de cinza
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Aplicar template matching
        res = cv2.matchTemplate(gray_frame, logo, cv2.TM_CCOEFF_NORMED)
        _, max_val, _, _ = cv2.minMaxLoc(res)

        # Se a correspondência for maior que o threshold, a logo foi detectada
        if max_val >= threshold:
            if not detection_started:
                # Marca o início da detecção
                start_time = frame_index / fps
                detection_started = True
        else:
            if detection_started:
                # Marca o fim da detecção
                end_time = frame_index / fps
                logo_timestamps.append(
                    {
                        "start": convert_seconds_to_timestamp(start_time),
                        "end": convert_seconds_to_timestamp(end_time),
                    }
                )
                detection_started = False

        frame_index += 1

    # Caso a logo esteja visível até o final do vídeo
    if detection_started:
        end_time = frame_index / fps
        logo_timestamps.append(
            {
                "start": convert_seconds_to_timestamp(start_time),
                "end": convert_seconds_to_timestamp(end_time),
            }
        )

    cap.release()

    # Salvar o resultado em um arquivo JSON
    # with open("logo_detection_result.json", "w") as json_file:
    #     json.dump(logo_timestamps, json_file, indent=4)

    # print("Detecção concluída. Resultados salvos em 'logo_detection_result.json'.")
    # print(logo_timestamps)
    return logo_timestamps

In [6]:
def detect_logo_in_video(video_path, logo_path, output_folder="tm_ccoeff_frames", threshold=0.6):
    # Abrir o vídeo e a logotipo target
    cap = cv2.VideoCapture(video_path)
    logo = cv2.imread(logo_path, 0)  # Lendo a logotipo em escala de cinza

    # Verifica se o vídeo foi aberto corretamente
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    # Criar pasta de saída se não existir
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obter fps para calcular o timestamp
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    detection_started = False
    logo_timestamps = []
    start_time = None
    end_time = None

    frame_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Converter frame para escala de cinza
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Aplicar template matching
        res = cv2.matchTemplate(gray_frame, logo, cv2.TM_CCOEFF_NORMED)
        _, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

        # Se a correspondência for maior que o threshold, a logo foi detectada
        if max_val >= threshold:
            if not detection_started:
                # Marca o início da detecção
                start_time = frame_index / fps
                detection_started = True

            # Desenhar um retângulo ao redor da logo detectada
            h, w = logo.shape
            top_left = max_loc
            bottom_right = (top_left[0] + w, top_left[1] + h)
            cv2.rectangle(frame, top_left, bottom_right, (0, 255, 0), 2)
        else:
            if detection_started:
                # Marca o fim da detecção
                end_time = frame_index / fps
                logo_timestamps.append(
                    {
                        "start": convert_seconds_to_timestamp(start_time),
                        "end": convert_seconds_to_timestamp(end_time),
                    }
                )
                detection_started = False

        # Salvar o frame como imagem
        frame_filename = os.path.join(output_folder, f"frame_{frame_index:05d}_grande.jpg")
        cv2.imwrite(frame_filename, frame)

        frame_index += 1

    # Caso a logo esteja visível até o final do vídeo
    if detection_started:
        end_time = frame_index / fps
        logo_timestamps.append(
            {
                "start": convert_seconds_to_timestamp(start_time),
                "end": convert_seconds_to_timestamp(end_time),
            }
        )

    cap.release()

    return logo_timestamps

In [7]:
video_path = "../videos/15s-canto-esquerdo-grande.mp4"
logo_path = "../target/logo_target.png"
detect_logo_in_video(video_path, logo_path)

[]